In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import concurrent.futures
import us
from ipumspy import readers
from scripts.clean_ipums import get_CPS
import plotly.graph_objects as go
import re

# Final Analysis for Research Brief

## Data

We will be requesting monthly CPS data from 1990 to 2024, with variables on economic and educational outcomes, as well as information on demographic characteristics, like race, geography, and disability and parental status. 

Note that this will take some time, given the number of datasets we're requesting from IPUMS. We're multithreading though, which should speed this up quite a bit.

In [ ]:
load_dotenv() # using a .env file
              # place your API key as IPUMS_API_KEY="xxx"
os.makedirs('finaldata',exist_ok=True) # make dir for files

my_vars = {'AGE', 'SEX', 'RACE', 'HISPAN', 'NCHILD', 
           'EMPSTAT', 'LABFORCE', 'SCHLCOLL', 'STATEFIP'}  # vars set
start,end = 1990,2024 # year range
# years < 2008 don't have 'DIFFANY', which causes exceptions when requesting data
vars_dict = {yr: my_vars if yr < 2008 
                         else my_vars | {'DIFFANY'} # add 'DIFFANY' to set
                                for yr in range(start,end+1)} 
#submit multiple requests
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as exe: 
    future_to_year = {exe.submit(get_CPS, # request function
                                 yr, # year
                                 vars_dict[yr], # variables for a given year
                                 f'NEET_{yr}', # file name
                                 'finaldata'): # data directory 
                                              yr for yr in range(start,end+1)} # single year CPS data request
    for future in concurrent.futures.as_completed(future_to_year):
        yr = future_to_year[future] # year to print exceptions
        try:
            result = future.result()
        except Exception as exc:
            print(f'Year {yr} generated an exception.')

Let's now load the data in. We can make some specifications which should speed this up a bit as well:
- only respondents aged 16-24 years-old
- only respondents not surveyed in June, July and August
- only respondents not part of the ASEC
- only respondents with recorded enrollment and employment statuses

In [20]:
# there's definitely a quicker way to do this, 
# but I wrote it quickly and it loads in about 5 min for me, fine for now
# will update at some point, if we need to use more cps data in the future
import warnings
warnings.filterwarnings(action = 'ignore', 
                        category = readers.CitationWarning) # catch warning output so output doesn't get cluttered

months = [i for i in range(1,13) if i not in [6,7,8]] # exclude summer months
timetrends = pd.DataFrame() # init empty df

for yr in range(start,end+1):
    ddi_name = os.path.join('finaldata',f'NEET_{yr}.xml')
    ddi = readers.read_ipums_ddi(ddi_name)

    dat_name = os.path.join('finaldata',f'NEET_{yr}.dat.gz')
    iter_df = readers.read_microdata_chunked(ddi,
                                             filename=dat_name,
                                             chunksize=10000)
    
    df = pd.concat([df[(df['AGE'] >= 16) &  
                           (df['AGE'] <= 24) & 
                           (df['MONTH'].isin(months)) &
                           ((df['ASECFLAG'] == 2) | (df['ASECFLAG'].isnull())) &
                           (df['EMPSTAT'].isin(range(10,37))) &
                           (df['SCHLCOLL'].isin(range(1,6)))] for df in iter_df],
                           ignore_index=True) # load in year df, with specifications
    timetrends = pd.concat(objs=(timetrends,
                           df),
                           ignore_index=True) # concat master df with yr df

In [21]:
timetrends['ASECFLAG'].value_counts(dropna=False)

ASECFLAG
<NA>    4094808
2        505280
Name: count, dtype: Int64

Let's quickly check to see if everything looks right.

In [22]:
timetrends['YEAR'].value_counts().sort_index()

YEAR
1990    167253
1991    162629
1992    156885
1993    151236
1994    146105
1995    140486
1996    125573
1997    126238
1998    125980
1999    127317
2000    128052
2001    137020
2002    147484
2003    146914
2004    145181
2005    144497
2006    144000
2007    141154
2008    139853
2009    140537
2010    140636
2011    137225
2012    133728
2013    131956
2014    131935
2015    129468
2016    127044
2017    123214
2018    116837
2019    110647
2020    100619
2021     97712
2022     92702
2023     90768
2024     91203
Name: count, dtype: Int64

In [23]:
timetrends['MONTH'].value_counts().sort_index()

MONTH
1     515986
2     512933
3     505280
4     513984
5     512350
9     510744
10    511733
11    510247
12    506831
Name: count, dtype: Int64

In [24]:
timetrends['AGE'].value_counts().sort_index()

AGE
16    573079
17    566895
18    521789
19    487507
20    483386
21    486910
22    488240
23    492329
24    499953
Name: count, dtype: Int64

Okay, that all looks good. Let's map our variables now.

In [25]:
# NEET
timetrends['NEET'] = 'not_neet'
timetrends.loc[(timetrends['EMPSTAT'].isin(range(20,37))) & (timetrends['SCHLCOLL'] == 5), 'NEET'] = 'neet' # NEET conditions
# NLE (NEET, but now only those not in school and not in labor force)
timetrends['NLE'] = 'not_neet'
timetrends.loc[(timetrends['LABFORCE'] == 1) & (timetrends['SCHLCOLL'] == 5), 'NLE'] = 'neet'
#Gender
gender_codes = {1 : 'men', 2 : 'women'} 
timetrends['sex'] = timetrends['SEX'].map(gender_codes)
# Race
race_codes = [
    ((timetrends['RACE'] == 300) & (timetrends['HISPAN'] == 0)),
    ((timetrends['RACE'] == 100) & (timetrends['HISPAN'] == 0)),
    ((timetrends['RACE'] == 200) & (timetrends['HISPAN'] == 0)),
    ((timetrends['RACE'].isin(range(650, 653))) & (timetrends['HISPAN'] == 0)),
    ((timetrends['HISPAN'] > 0) & (timetrends['HISPAN'] < 902))
]
race_choices = ['aian', 'white', 'black', 'asian', 'hispanic']
timetrends['race_cat'] = np.select(race_codes, race_choices, default = 'other')
# State
state_names = us.states.mapping('fips', 'name') # dictionary with STATEFIP as key
state_names['11'] = 'District of Columbia' # DC isn't included, so manually add it
timetrends['state'] = timetrends['STATEFIP'].astype(str).str.zfill(2) 
timetrends['state'] = timetrends['state'].map(state_names)
# Kids
child_dict = {i:'has_child' if i > 0 else 'no_child' for i in range(10)}
timetrends['kids'] = timetrends['NCHILD'].map(child_dict)
# Disability
disability_dict = {1 : 'no_dis', 2 : 'has_dis'}
timetrends['dis'] = timetrends['DIFFANY'].map(disability_dict)
# Looking for work status
timetrends['looking_for_work'] = ''
timetrends.loc[timetrends['EMPSTAT'].isin(range(10,13)), 'looking_for_work'] = 'employed'
timetrends.loc[timetrends['EMPSTAT'].isin(range(20,23)), 'looking_for_work'] = 'looking'
timetrends.loc[timetrends['EMPSTAT'].isin(range(30,37)), 'looking_for_work'] = 'not_looking'

Let's get our NEET calculator function now.

In [26]:
def get_neet(df=timetrends,
             query_str=None,
             groupings=[]):
    '''returns NEET rates'''
    if query_str:
        df = df.query(query_str)
    agg_wt = df.groupby(groupings)['WTFINL'].sum().unstack() # agg sum
    agg_wt= agg_wt.eval('neet_rate = neet / (neet + not_neet) * 100')['neet_rate'] # NEET rates
    return agg_wt

## Analysis

### 1. Overall Time Trend

In [27]:
overall = get_neet(groupings=['YEAR', 'sex', 'NEET']).reindex().unstack()
overall

sex,men,women
YEAR,,
1990,10.017256,18.554956
1991,11.501997,19.691814
1992,11.518769,18.982120
1993,10.825050,18.303880
1994,10.695621,17.644251
1995,10.512726,17.586979
1996,10.425318,16.629541
1997,10.019295,15.197452
1998,9.366412,14.474526


In [28]:
fig = go.Figure()
colors = {'men':'skyblue','women':'pink'}
for i in ['men', 'women']:
    fig.add_trace(go.Scatter(
    x=overall.index, y=overall[i].round(2),
    mode='lines+markers',
    name=i.title(),
    line={
        'color': colors[i],
        'width': 3
    }
))
fig.update_layout(
    font={'family': 'Georgia, serif'},
    title={'text': '<b>NEET rates for 16-24 y.o. (1990-2024)</b>',
           'subtitle': {'text': '(<b>excluding summer months</b>)'}},
    xaxis={'title':{'text': '<b>Year'}},
    yaxis={'title':{'text': '<b>NEET rate (%)'}},
    legend = {'orientation': 'h'},
    width=650, height=400,
    template='plotly_white',
    margin={'pad': 0, 'b': 20, 'l': 20, 'r': 20, 't': 100}
)
fig.update_yaxes(range=[5,25])
fig.show()

### 2. By Disability and Parental Status

#### Disability

In [29]:
disability = get_neet(groupings=['YEAR','dis', 'sex', 'NEET']).unstack().unstack()
disability

sex         men                 women           
dis     has_dis     no_dis    has_dis     no_dis
YEAR                                            
2008  35.383201  11.490264  31.522891  13.810852
2009  37.390557  13.507928  36.844796  14.496779
2010  36.908343  13.968382  37.937467  14.622856
2011  41.337963  12.571788  36.292516  14.421134
2012  39.829024  11.959576  36.648988  13.910660
2013  40.311276  12.823393  39.476145  14.720988
2014  38.802650  11.920432  37.664481  13.938124
2015  38.138646  10.994551  36.490799  13.053425
2016  36.591790  10.609934  38.695747  12.024602
2017  37.037077   9.638086  33.962683  11.201026
2018  36.580314   9.738044  31.501278  11.414741
2019  35.576955   9.563309  35.431805  10.717032
2020  38.556539  13.126573  33.527831  13.891694
2021  36.196981  11.727650  36.001684  12.384964
2022  34.454245  10.481587  32.366725  11.802840
2023  33.080058  10.351587  29.401336  11.237574
2024  35.375930  10.571921  30.627373  11.545734

In [30]:
fig = go.Figure()
colors_dis = {('men', 'has_dis') : 'darkblue', ('men', 'no_dis'): 'teal',
          ('women', 'has_dis'): 'darkred', ('women', 'no_dis'): 'salmon'}
for i in [('men', 'has_dis'), ('men', 'no_dis'),
          ('women', 'has_dis'), ('women', 'no_dis')]:
    fig.add_trace(go.Scatter(
    x=disability.index, y=disability[i].round(2),
    mode='lines+markers',
    name=str(i),
    line={
        'color': colors_dis[i],
        'width': 3
    }
))
fig.update_layout(
    font={'family': 'Georgia, serif'},
    title={'text': '<b>NEET rates for 16-24 y.o. by disability status (2008-2024)</b>',
           'subtitle': {'text': '(<b>excluding summer months</b>)'}},
    xaxis={'title':{'text': '<b>Year'}}, yaxis={'title':{'text': '<b>NEET rate (%)'}},
    legend = {'orientation': 'v', 'title': {'text': '<b>group</b><br>(click to toggle lines)'}},
    width=700, height=450,
    template='plotly_white',
    margin={'pad': 0, 'b': 20, 'l': 20, 'r': 20, 't': 100}
)
fig.update_yaxes(range=[0,45])
fig.show()

#### Parent Status

In [31]:
parents = get_neet(groupings=['YEAR','kids', 'sex', 'NEET']).unstack().unstack()
parents

sex         men                 women           
kids  has_child   no_child  has_child   no_child
YEAR                                            
1990  12.682835   9.825847  51.240403  10.047968
1991  14.447692  11.297857  53.760222  10.758971
1992  15.510384  11.238480  51.506941  10.384993
1993  14.324418  10.574446  49.096433  10.062685
1994  13.164635  10.511311  46.836758   9.672197
1995  13.748654  10.247533  45.963538   9.667492
1996  13.434661  10.168428  42.756678   9.589749
1997  12.738879   9.815989  39.909948   9.006265
1998  11.762226   9.185394  38.724209   8.450738
1999  12.242810   9.057339  38.108430   8.660436
2000  10.991487   9.080354  36.892527   8.775162
2001  13.872129  10.070278  38.544966   8.962057
2002  15.528324  10.726495  39.230139   9.893444
2003  14.638342  10.952156  40.443888   9.634876
2004  14.399951  10.572821  41.535737   9.241213
2005  14.485754  10.545435  40.780951   9.245385
2006  13.768809   9.785099  41.201042   8.989727
2007  14.522553  10.119130  40.896221   9.006624
2008  16.835747  11.449983  41.455205   9.540354
2009  21.702238  14.118808  42.281555  10.487065
2010  23.457488  14.442232  42.095624  10.473409
2011  19.692112  13.388415  41.712388  10.302191
2012  17.476849  12.839783  39.782238  10.401961
2013  19.575034  13.739369  41.990175  11.220636
2014  17.314039  12.842414  41.081240  10.709002
2015  13.871704  12.078354  38.270847  10.372872
2016  13.086532  11.710613  37.311197   9.862926
2017  12.425278  10.760555  38.252397   8.949913
2018  13.329336  10.848087  37.518641   9.372131
2019  12.308579  10.792847  36.818869   9.036978
2020  17.263358  14.261671  42.923878  12.164153
2021  14.890822  12.926447  42.631519  10.938475
2022  11.593876  11.789582  40.288790  10.487357
2023  10.779920  11.582569  39.238638  10.089062
2024  15.414303  11.970941  39.870786  10.725570

In [32]:
fig = go.Figure()
colors_parents = {('men', 'has_child') : 'darkblue', ('men', 'no_child'): 'teal',
          ('women', 'has_child'): 'darkred', ('women', 'no_child'): 'salmon'}
for i in [('men', 'has_child'), ('men', 'no_child'),
          ('women', 'has_child'), ('women', 'no_child')]:
    fig.add_trace(go.Scatter(
    x=parents.index, y=parents[i].round(2),
    mode='lines+markers',
    name=str(i),
    line={
        'color': colors_parents[i],
        'width': 3
    }
))
fig.update_layout(
    font={'family': 'Georgia, serif'},
    title={'text': '<b>NEET rates for 16-24 y.o. by parental status (1990-2024)</b>',
           'subtitle': {'text': '(<b>excluding summer months</b>)'}},
    xaxis={'title':{'text': '<b>Year'}}, yaxis={'title':{'text': '<b>NEET rate (%)'}},
    legend = {'orientation': 'v', 'title': {'text': '<b>group</b><br>(click to toggle lines)'}},
    width=700, height=450,
    template='plotly_white',
    margin={'pad': 0, 'b': 20, 'l': 20, 'r': 20, 't': 100}
)
fig.update_yaxes(range=[0,55])
fig.show()

### 3. By Race

In [33]:
race = get_neet(groupings=['YEAR', 'race_cat', 'sex', 'NEET']).unstack().unstack()
race

sex             men                                                         \
race_cat       aian      asian      black   hispanic      other      white   
YEAR                                                                         
1990      17.792061   6.088405  19.424460  12.196839   8.286094   7.851776   
1991      16.483241   7.679942  21.526662  14.435081  11.583538   9.019072   
1992      17.195371   7.144802  22.002682  15.105539  14.590362   8.787811   
1993      14.737228   6.680116  20.802851  13.984624  11.611785   8.259422   
1994      15.303534   6.242215  19.754737  13.545420  13.252633   8.356211   
1995      17.434698   5.826515  19.904843  14.114436   9.602006   7.968466   
1996      17.679509   6.999938  19.731376  13.625734  17.360115   7.877675   
1997      18.918615   6.412437  21.206018  12.417704  14.274826   7.161822   
1998      17.842630   7.631807  18.444077  11.843369  10.421197   6.889832   
1999      18.678590   7.229050  17.881400  11.552977   8.890147   6.989450   
2000      16.133279   6.236770  18.653547  10.188148  13.028117   6.953599   
2001      18.632717   7.505529  20.082687  10.940845   8.249167   8.160651   
2002      23.246373   8.100413  20.170965  12.485824  10.388540   8.681938   
2003      27.786166   8.090233  20.185410  12.412850  13.075918   8.929568   
2004      22.071732   6.112682  18.297314  12.720929  14.297244   8.775184   
2005      21.559145   7.017076  20.310462  12.040403  13.607423   8.439134   
2006      22.380038   8.388060  17.758050  10.053234  12.062284   8.237464   
2007      25.028912   6.684778  17.198938  11.692203   9.973620   8.535765   
2008      22.544240   6.934249  18.325694  13.041100  15.178085   9.969911   
2009      23.042817   7.578076  22.114607  16.039941  17.909627  12.575396   
2010      27.443900  10.226637  21.620234  16.471759  15.127733  13.024456   
2011      27.304922   8.744732  21.342644  14.812804  15.465041  11.641200   
2012      23.276190   7.289460  20.835278  14.204371  14.721758  11.006058   
2013      26.158203   8.995392  21.948544  14.546924  14.916900  12.095200   
2014      20.143735   8.468240  21.338119  12.668237  13.680895  11.382013   
2015      26.510248   8.050023  18.805225  12.810741  12.101768  10.391904   
2016      26.792631   6.965542  17.940867  12.778545  10.056157  10.112268   
2017      25.394327   7.887718  17.136651  11.161252  10.787912   9.147254   
2018      23.186782   7.897839  16.864422  11.290861  12.260323   9.348466   
2019      26.155855   6.253714  16.640603  10.808921  10.648598   9.675243   
2020      27.441211  11.256802  20.773689  15.381309  14.366293  12.450649   
2021      28.116149   9.687066  17.897574  14.034107  13.522494  11.387559   
2022      21.942822   9.407318  15.681923  12.331524  11.200503  10.685611   
2023      22.131887   8.145690  17.181944  11.937083  11.152459  10.195293   
2024      18.535947   8.018668  17.729826  13.460164  13.753342  10.158650   

sex           women                                                         
race_cat       aian      asian      black   hispanic      other      white  
YEAR                                                                        
1990      28.946565  12.246194  28.631100  30.824047  12.052036  14.283629  
1991      32.591962  14.559935  30.000238  31.563147  18.751884  15.223325  
1992      31.072298  13.863800  29.498148  30.691338  25.411884  14.387876  
1993      32.492373  13.758611  27.065046  31.834467  15.510439  13.555402  
1994      26.106834  11.244947  25.016533  31.537457  22.784708  13.417921  
1995      21.210123  13.481402  23.902497  31.889579  15.395688  13.492568  
1996      21.834056  12.408812  23.217990  29.969848  10.380543  12.530164  
1997      28.796965  11.422062  21.387577  25.886326  18.090190  11.533193  
1998      28.058792   8.663242  19.246553  25.443863  20.757087  11.072472  
1999      22.734040   9.188241  18.455769  26.193738  11.876421  11.047331  
2000      21.916387   9.633709  18

In [34]:
fig = go.Figure()
# colors_parents = {('men', 'has_child') : 'darkblue', ('men', 'no_child'): 'teal',
#           ('women', 'has_child'): 'darkred', ('women', 'no_child'): 'salmon'}
race_cols = [(s,r) for s in ['men','women'] for r in ['white','black','asian','hispanic']]
for i in race_cols:
    fig.add_trace(go.Scatter(
    x=race.index, y=race[i].round(2),
    mode='lines+markers',
    name=str(i),
    line={
        'width': 3
    }
))
fig.update_layout(
    font={'family': 'Georgia, serif'},
    title={'text': '<b>NEET rates for 16-24 y.o. by race (1990-2024)</b>',
           'subtitle': {'text': '(<b>excluding summer months</b>)'}},
    xaxis={'title':{'text': '<b>Year'}}, yaxis={'title':{'text': '<b>NEET rate (%)'}},
    legend = {'orientation': 'v', 'title': {'text': '<b>group</b><br>(click to toggle lines)'}},
    width=800, height=500,
    template='plotly_white',
    margin={'pad': 0, 'b': 20, 'l': 20, 'r': 20, 't': 100}
)
fig.update_yaxes(range=[0,35])
fig.show()

### 4. By State

In [35]:
states = get_neet(groupings=['YEAR', 'state', 'sex', 'NEET']).unstack().unstack()
states

sex          men                                                         \
state    Alabama     Alaska    Arizona   Arkansas California   Colorado   
YEAR                                                                      
1990   17.789268  13.022989  10.059219  12.342674  10.690877   5.876648   
1991   11.663873  12.982497  12.172687  10.351329  11.448550  10.175277   
1992   12.384915  13.447323  12.435461   9.905128  13.086376   8.351902   
1993   11.685396  13.705490   9.523923  12.639078  12.891591  10.481227   
1994   13.639536  11.499796   9.396569  10.112075  11.630305   9.935657   
1995   13.732239  11.559305  10.584245   8.833456  11.349198   7.229902   
1996   12.198311  17.469593   8.719728   9.626303  10.843127   7.525818   
1997   10.101662  11.984272   8.228975  12.934259  10.473096   7.435272   
1998   10.409624  15.237285  10.162739  14.290726  10.715124   6.043558   
1999   11.527018  11.048588   9.279325  13.895766   9.771514   7.561349   
2000   11.560071  15.237825  10.275009  11.560486   8.714224   8.845808   
2001   10.480869  15.208823   7.484361  12.253817   9.737517   7.552288   
2002   10.923176  17.383830   9.993658   9.923590  11.241951   9.266082   
2003   13.944306  17.280968   9.016102  15.347888  10.383193   9.670630   
2004   14.342094  14.735696  13.466072  11.242054  10.660356  13.449040   
2005   13.070980  13.708188  13.125358  11.670341   9.507806   9.348914   
2006   10.749749  13.603772  12.377457  10.085315   9.681981   7.791256   
2007   15.846194  13.600774  14.108770   8.809086   9.940758   7.614220   
2008   16.455394  16.901525  15.040700  12.038594  12.285823   9.730946   
2009   16.569170  15.112310  15.599763  13.888063  14.903146  10.974313   
2010   20.878563  16.784127  17.274507  15.337506  14.956209  11.480291   
2011   20.260446  17.017917  12.513414  12.340872  13.446204  10.385274   
2012   13.764159  15.607704  12.861286  17.278000  13.792601  11.878875   
2013   18.489113  12.998826  15.207531  17.339858  13.626322  12.130996   
2014   16.383112  17.449321  14.660296  12.102985  11.753354   8.105108   
2015   15.469984  18.077538  13.272370  12.215959  11.804094   7.392205   
2016   14.689129  16.321520  12.641763  10.611146  11.333316   7.509258   
2017   14.367545  17.421942  11.152021  12.142305  10.798235   9.581403   
2018   11.254147  21.212086  13.575823  11.548364  10.566025   7.480182   
2019   13.672235  21.584504  12.255686   8.105634  10.894487   4.997040   
2020   13.998650  22.854909  14.942274  14.585685  14.713730  12.195227   
2021   11.446704  20.795798  15.107248  13.658519  14.152310  13.701161   
2022   11.712673  21.566225  10.026272  13.801662  11.885961  10.768861   
2023   10.838755  19.228313   8.816277  14.989986  12.138906   8.299743   
2024   12.610009  19.608925  13.165344  13.719843  12.317259   9.430356   

sex                                                           ...  \
state Connecticut   Delaware District of Columbia    Florida  ...   
YEAR                                                          ...   
1990     9.112945  10.004748            12.339956   9.604940  ...   
1991     8.094354  13.335176            16.793386  10.194651  ...   
1992    12.344283   8.705180            18.748204  10.899828  ...   
1993     6.910364  10.468923            21.047283  10.288782  ...   
1994     6.684156   8.525706            17.346486  11.951006  ...   
1995     7.908366  10.757934            18.863394  11.679286  ...   
1996     8.352465  11.013097            16.893991  10.676622  ...   
1997    10.723387   7.900764            18.669108   9.628984  ...   
1998    10.929228   9.015923            19.117288   8.321830  ...   
1999     9.957125   9.568248            13.592445   8.413869  ...   
2000     7.154726   9.133627            14.753425   8.992537  ...   
2001     7.466993  11.704323            16.404364  10.156836  ...   
2002     8.590988   9.361183            14.101565  11.538290  ...   
2003    10.141778   8.192819            12.55

In [36]:
# let's just plot a couple states
from plotly.subplots import make_subplots
states_and_coords = {'California': (1,1),'New York': (1,2),
                     'West Virginia': (2,1),'Alaska': (2,2)}
fig = make_subplots(rows=2,cols=2, subplot_titles=tuple(states_and_coords.keys()))
for i in states_and_coords.keys():
    fig.add_trace(go.Scatter(
    x=states.index, y=states[('men', i)].round(2),
    mode='lines+markers',
    name='Men',
    line={
        'width': 3, 'color': 'skyblue'
    }
),
row=states_and_coords[i][0],col=states_and_coords[i][1])
    
    fig.add_trace(go.Scatter(
    x=states.index, y=states[('women', i)].round(2),
    mode='lines+markers',
    name='Women',
    line={
        'width': 3, 'color': 'pink'
    }
),
row=states_and_coords[i][0],col=states_and_coords[i][1])
fig.update_layout(
    font={'family': 'Georgia, serif'},
    title={'text': '<b>NEET rates for 16-24 y.o. by state (1990-2024)</b>',
           'subtitle': {'text': '(<b>excluding summer months</b>)'}},
    showlegend=False,
    width=750, height=500,
    template='plotly_white',
    margin={'pad': 0, 'b': 20, 'l': 20, 'r': 20, 't': 100}
)
fig.update_yaxes(range=[0,35])
fig.show()

### 5. By NEET Status

Here, we'll break down the male NEETs by those who are looking for work, and those who are not looking for work.

In [54]:
weighted_sums = timetrends.query('sex == "men"').groupby('YEAR')['WTFINL'].sum()
neets_work = timetrends.query('sex == "men" and NEET == "neet"').copy()
neets_work = neets_work.groupby(['YEAR','looking_for_work'])['WTFINL'].sum().unstack()
neets_work['looking_rate'] = neets_work['looking'] / weighted_sums * 100
neets_work['not_looking_rate'] = neets_work['not_looking'] / weighted_sums * 100
neets_work = neets_work.reindex(columns=['looking_rate', 'not_looking_rate'])
neets_work

looking_for_work,looking_rate,not_looking_rate
YEAR,,
1990,5.561085,4.456172
1991,6.602031,4.899966
1992,6.871597,4.647172
1993,6.232610,4.592440
1994,5.646683,5.048937
1995,5.328408,5.184318
1996,5.345382,5.079936
1997,4.847548,5.171746
1998,4.355443,5.010969


In [67]:
neets_work_dict = {'not_looking_rate': 'darkblue','looking_rate': 'lightblue'}
fig = go.Figure()
for col in neets_work_dict.keys():
    fig.add_trace(go.Scatter(
        x=neets_work.index, y=neets_work[col].round(2),
        name=col,
        mode='lines',
        stackgroup='one',
        line_color=neets_work_dict[col]
    ))
fig.update_layout(
    font={'family': 'Georgia, serif'},
    title={'text': '<b>Male NEET rates for 16-24 y.o. by Labor Force Status (1990-2024)</b>',
           'subtitle': {'text': '(<b>excluding summer months</b>)'}},
    legend = {'orientation': 'v', 'title': {'text': '<b>group</b><br>(click to toggle lines)'}},
    xaxis={'title':{'text': '<b>Year'}}, yaxis={'title':{'text': '<b>NEET rate (%)'}},
    width=750, height=500,
    template='plotly_white',
    margin={'pad': 0, 'b': 20, 'l': 20, 'r': 20, 't': 100}
)
fig.update_yaxes(range=[0,20])

### 6. 2024 Compositions

In this section, we break down NEET and non-NEET men and women by whether they're seeking work or not, and if they're disabled or parents.

In [211]:
looking_comp = timetrends.query('YEAR==2024 and ' \
                                'looking_for_work=="looking"').groupby(['NEET',
                                                                        'looking_for_work','sex'])['WTFINL'].sum().unstack()
looking_comp['group'] = looking_comp.index
not_looking_comp = timetrends.query('YEAR==2024 and ' \
                                'looking_for_work=="not_looking"').groupby(['dis','kids',
                                                                            'NEET','looking_for_work','sex'])['WTFINL'].sum().unstack()
not_looking_comp['group'] = not_looking_comp.index
comp_merged = pd.concat([looking_comp,not_looking_comp],ignore_index=True)
comp_merged['group'] = comp_merged['group'].astype(str).str.replace(r'\(|\)|,|\'','',regex=True).str.replace(' ',',')
comp_merged

sex,men,women,group
0,6.352238e+06,4.516327e+06,"neet,looking"
1,2.978539e+06,2.694375e+06,"not_neet,looking"
2,4.474034e+04,2.267136e+05,"has_dis,has_child,neet,not_looking"
3,2.348102e+04,4.532129e+04,"has_dis,has_child,not_neet,not_looking"
4,3.235481e+06,2.150068e+06,"has_dis,no_child,neet,not_looking"
5,3.462376e+06,2.556237e+06,"has_dis,no_child,not_neet,not_looking"
6,3.002784e+05,3.581211e+06,"no_dis,has_child,neet,not_looking"
7,1.628664e+05,7.205781e+05,"no_dis,has_child,not_neet,not_looking"
8,1.137853e+07,1.130911e+07,"no_dis,no_child,neet,not_looking"
9,5.962024e+07,5.798990e+07,"no_dis,no_child,not_neet,not_looking"


For now, let's just plot the NEETs.

In [255]:
neets_col = [grp for grp in comp_merged['group'] if 'not_neet' not in grp]
neets_comp = comp_merged.loc[comp_merged['group'].isin(neets_col)].copy()
neets_comp['men'] = (neets_comp['men'] / 
                     timetrends.query('YEAR==2024 and sex=="men" and NEET=="neet"')['WTFINL'].sum()) * 100
neets_comp['women'] = (neets_comp['women'] / 
                     timetrends.query('YEAR==2024 and sex=="women" and NEET=="neet"')['WTFINL'].sum()) * 100

fig = go.Figure()
for nm in neets_comp['group']:
    nm2 = nm.replace(',','<br>')
    fig.add_trace(go.Bar(y=neets_comp.loc[neets_comp['group']==nm, 'men'],
                         x=[f'({nm2}<br><b>men</b>)']))
    fig.add_trace(go.Bar(y=neets_comp.loc[neets_comp['group']==nm, 'women'],
                         x=[f'({nm2}<br><b>women</b>)']))
fig.update_layout(
    font={'family': 'Georgia, serif'},
    title={'text': '<b>NEET composition for 16-24 y.o. (2024)</b>',
           'subtitle': {'text': '(<b>excluding summer months</b>)'}},
    showlegend=False,
    xaxis={'title':{'text': '<b>Group'}}, yaxis={'title':{'text': '<b>Within-Gender Share (%)'}},
    width=1000, height=600,
    template='plotly_white',
    margin={'pad': 0, 'b': 20, 'l': 20, 'r': 20, 't': 50}
)
fig.show()